##### Copyright 2021 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TF-Agents로 DQN(Deep Q Network) 훈련하기

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/1_dqn_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>
</td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/agents/blob/master/docs/tutorials/1_dqn_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     View source on GitHub</a>
  </td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/agents/docs/tutorials/1_dqn_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

## 소개


이 예제는 TF-Agents 라이브러리를 사용하여 Cartpole 환경에서 [DQN(Deep Q Networks)](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf) 에이전트를 훈련하는 방법을 보여줍니다.

![카트 폴 환경](https://raw.githubusercontent.com/tensorflow/agents/master/docs/tutorials/images/cartpole.png)

훈련, 평가 및 데이터 수집을 위한 RL(Reinforcement Learning) 파이프라인의 모든 구성 요소를 설명합니다.

이 코드를 실시간으로 실행하려면, 위의 'Google Colab에서 실행' 링크를 클릭하세요.


## 설정

다음 종속성을 설치하지 않은 경우, 다음을 실행합니다.

In [ ]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet

In [ ]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.specs import tensor_spec
from tf_agents.utils import common

In [ ]:
# Set up a virtual display for rendering OpenAI gym environments.
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

In [ ]:
tf.version.VERSION

## 하이퍼 매개변수

In [ ]:
num_iterations = 20000 # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
collect_steps_per_iteration =   1# @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}

batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}

num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}

## 환경

강화 학습(RL)에서 환경은 해결해야 할 작업 또는 문제를 나타냅니다. `tf_agents.environments` 도구 모음을 사용하여 TF-Agents에서 표준 환경을 만들 수 있습니다. TF-Agents에는 OpenAI Gym, Atari 및 DM Control과 같은 소스에서 환경을 로드하기 위한 도구 모음이 있습니다.

OpenAI Gym 도구 모음에서 CartPole 환경을 로드합니다. 

In [ ]:
env_name = 'CartPole-v0'
env = suite_gym.load(env_name)

이 환경을 렌더링하여 형태를 볼 수 있습니다. 자유로 흔들리는 막대가 카트에 부착되어 있습니다. 목표는 막대가 위를 향하도록 카트를 오른쪽이나 왼쪽으로 이동하는 것입니다.

In [ ]:
#@test {"skip": true}
env.reset()
PIL.Image.fromarray(env.render())

`environment.step` 메서드는 환경에서 `action`을 사용하여 환경의 다음 관찰 값과 행동에 대한 보상이 포함된 `TimeStep` 튜플을 반환합니다.

`time_step_spec ()` 메서드는 `TimeStep` 튜플의 사양을 반환합니다. `observation` 속성은 관찰 값의 형상, 데이터 유형 및 허용되는 값의 범위를 보여줍니다. `reward` 속성은 보상에 대해 같은 세부 사항을 보여줍니다.


In [ ]:
print('Observation Spec:')
print(env.time_step_spec().observation)

In [ ]:
print('Reward Spec:')
print(env.time_step_spec().reward)

`action_spec()` 메서드는 형상, 데이터 유형 및 유효한 행동의 허용된 값을 반환합니다.

In [ ]:
print('Action Spec:')
print(env.action_spec())

Cartpole 환경에서:

- `observation`은 4개의 부동 소수점으로 구성된 배열입니다.
    - 카트의 위치와 속도
    - 막대의 각도 위치와 속도
- `reward`는 스칼라 부동 소수점 값입니다.
- `action`은 두 개의 가능한 값만 있는 스칼라 정수입니다.
    - `0` — "왼쪽으로 이동"
    - `1` — "오른쪽으로 이동"


In [ ]:
time_step = env.reset()
print('Time step:')
print(time_step)

action = np.array(1, dtype=np.int32)

next_time_step = env.step(action)
print('Next time step:')
print(next_time_step)

일반적으로 훈련용과 평가용의 두 가지 환경이 인스턴스화됩니다. 

In [ ]:
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

Cartpole 환경은 대부분의 환경과 마찬가지로 순수한 Python으로 작성됩니다. `TFPyEnvironment` 래퍼를 사용하여 TensorFlow로 변환됩니다.

원래 환경의 API는 Numpy 배열을 사용합니다. `TFPyEnvironment`는 Tensorflow 에이전트 및 정책과 호환되도록 Numpy 배열을 `Tensors`로 변환합니다.


In [ ]:
train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

## 에이전트

RL 문제를 해결하는 데 사용되는 알고리즘은 `Agent`로 표시됩니다. TF-Agents는 다음을 포함하여 다양한 `Agents`의 표준 구현을 제공합니다.

- [DQN](https://storage.googleapis.com/deepmind-media/dqn/DQNNaturePaper.pdf)(이 튜토리얼에서 사용)
- [강화](https://www-anw.cs.umass.edu/~barto/courses/cs687/williams92simple.pdf)
- [DDPG](https://arxiv.org/pdf/1509.02971.pdf)
- [TD3](https://arxiv.org/pdf/1802.09477.pdf)
- [PPO](https://arxiv.org/abs/1707.06347)
- [SAC](https://arxiv.org/abs/1801.01290)

DQN 에이전트는 불연속 행동 공간이 있는 모든 환경에서 사용할 수 있습니다.

DQN 에이전트의 핵심은 `QNetwork`이며, 환경의 주어진 관찰 값을 고려하여 모든 행동에 대한 `QValues`(기대 이익)를 학습하여 예측할 수 있는 신경망 모델입니다.

We will use `tf_agents.networks.` to create a `QNetwork`. The network will consist of a sequence of `tf.keras.layers.Dense` layers, where the final layer will have 1 output for each possible action.

In [ ]:
fc_layer_params = (100, 50)
action_tensor_spec = tensor_spec.from_spec(env.action_spec())
num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

# Define a helper function to create Dense layers configured with the right
# activation and kernel initializer.
def dense_layer(num_units):
  return tf.keras.layers.Dense(
      num_units,
      activation=tf.keras.activations.relu,
      kernel_initializer=tf.keras.initializers.VarianceScaling(
          scale=2.0, mode='fan_in', distribution='truncated_normal'))

# QNetwork consists of a sequence of Dense layers followed by a dense layer
# with `num_actions` units to generate one q_value per available action as
# its output.
dense_layers = [dense_layer(num_units) for num_units in fc_layer_params]
q_values_layer = tf.keras.layers.Dense(
    num_actions,
    activation=None,
    kernel_initializer=tf.keras.initializers.RandomUniform(
        minval=-0.03, maxval=0.03),
    bias_initializer=tf.keras.initializers.Constant(-0.2))
q_net = sequential.Sequential(dense_layers + [q_values_layer])

이제 `tf_agents.agents.dqn.dqn_agent`를 사용하여 `DqnAgent`를 인스턴스화합니다 . 또한, `time_step_spec`, `action_spec` 및 QNetwork 외에도, 에이전트 생성자는 옵티마이저(이 경우, `AdamOptimizer`), 손실 함수 및 정수 step counter가 필요합니다.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()

## 정책

정책은 에이전트가 환경에서 행동하는 방식을 정의합니다. 일반적으로 강화 학습의 목표는 정책이 원하는 결과를 얻을 때까지 기본 모델을 훈련하는 것입니다.

이 튜토리얼에서:

- 원하는 결과는 카트 위에서 막대의 균형을 똑바로 유지하는 것입니다.
- 정책은 각 `time_step` 관찰 값에 대한 행동(왼쪽 또는 오른쪽)을 반환합니다.

에이전트에는 두 가지 정책이 있습니다.

- `agent.policy` — 평가 및 배포에 사용되는 기본 정책입니다.
- `agent.collect_policy` — 데이터 수집에 사용되는 두 번째 정책입니다.


In [ ]:
eval_policy = agent.policy
collect_policy = agent.collect_policy

에이전트와 독립적으로 정책을 만들 수 있습니다. 예를 들어, `tf_agents.policies.random_tf_policy`를 사용하여 각 `time_step` 동안 행동을 무작위로 선택하는 정책을 생성합니다.

In [ ]:
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

정책에서 행동을 얻으려면 `policy.action(time_step)` 메서드를 호출합니다. `time_step`에는 환경의 관찰 값이 포함됩니다. 이 메서드는 `PolicyStep`를 반환하는데, 이는 3가지 구성 요소를 가진 명명된 튜플입니다.

- `action` — 취할 행동(이 경우, `0` 또는 `1` )
- `state` — 상태 저장(즉, RNN 기반) 정책에 사용됨
- `info` — 활동의 로그 확률과 같은 보조 데이터

In [ ]:
example_environment = tf_py_environment.TFPyEnvironment(
    suite_gym.load('CartPole-v0'))

In [ ]:
time_step = example_environment.reset()

In [ ]:
random_policy.action(time_step)

## 메트릭 및 평가

정책을 평가하는 데 사용되는 가장 일반적인 메트릭은 평균 이익입니다. 이익은 에피소드의 환경에서 정책을 실행하는 동안 얻은 보상의 합계입니다. 여러 에피소드가 실행되어 평균 이익이 산출됩니다.

다음 함수는 정책, 환경 및 여러 에피소드를 고려하여 정책의 평균 이익을 계산합니다.


In [ ]:
#@test {"skip": true}
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


# See also the metrics module for standard implementations of different metrics.
# https://github.com/tensorflow/agents/tree/master/tf_agents/metrics

`random_policy`에서 이 계산을 실행하면 환경의 기준 성능이 표시됩니다.

In [ ]:
compute_avg_return(eval_env, random_policy, num_eval_episodes)

## 재현 버퍼

In order to keep track of the data collected from the environment, we will use [Reverb](https://deepmind.com/research/open-source/Reverb), an efficient, extensible, and easy-to-use replay system by Deepmind. It stores experience data when we collect trajectories and is consumed during training.

This replay buffer is constructed using specs describing the tensors that are to be stored, which can be obtained from the agent using agent.collect_data_spec.


In [ ]:
table_name = 'uniform_table'
replay_buffer_signature = tensor_spec.from_spec(
      agent.collect_data_spec)
replay_buffer_signature = tensor_spec.add_outer_dim(
    replay_buffer_signature)

table = reverb.Table(
    table_name,
    max_size=replay_buffer_max_length,
    sampler=reverb.selectors.Uniform(),
    remover=reverb.selectors.Fifo(),
    rate_limiter=reverb.rate_limiters.MinSize(1),
    signature=replay_buffer_signature)

reverb_server = reverb.Server([table])

replay_buffer = reverb_replay_buffer.ReverbReplayBuffer(
    agent.collect_data_spec,
    table_name=table_name,
    sequence_length=2,
    local_server=reverb_server)

rb_observer = reverb_utils.ReverbAddTrajectoryObserver(
  replay_buffer.py_client,
  table_name,
  sequence_length=2)

에이전트 대부분의 경우, `collect_data_spec`은 `Trajectory`라는 명명된 튜플이며 관찰 값, 행동, 보상 및 기타 항목에 대한 사양을 포함합니다.

In [ ]:
agent.collect_data_spec

In [ ]:
agent.collect_data_spec._fields

## 데이터 수집

이제 환경에서 임의의 정책을 몇 단계 동안 실행하여 재현 버퍼에 데이터를 기록합니다.

Here we are using 'PyDriver' to run the experience collecting loop. You can learn more about TF Agents driver in our [drivers tutorial](https://www.tensorflow.org/agents/tutorials/4_drivers_tutorial).

In [ ]:
#@test {"skip": true}
py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      random_policy, use_tf_function=True),
    [rb_observer],
    max_steps=initial_collect_steps).run(train_py_env.reset())

재현 버퍼는 Trajectory의 모음입니다.

In [ ]:
# For the curious:
# Uncomment to peel one of these off and inspect it.
# iter(replay_buffer.as_dataset()).next()

에이전트는 재현 버퍼에 액세스해야 합니다. 에이전트에 데이터를 공급하는 반복 가능한 `tf.data.Dataset` 파이프라인을 작성하면 됩니다.

재현 버퍼의 각 행은 단일 관찰 단계만 저장합니다. 그러나 DQN 에이전트는 손실을 계산하기 위해 현재 및 다음 관측 값이 모두 필요하므로 데이터세트 파이프라인은 배치의 각 항목에 대해 두 개의 인접한 행을 샘플링합니다(`num_steps=2`).

이 데이터세트는 병렬 호출을 실행하고 데이터를 프리페치하여 최적화됩니다.

In [ ]:
# Dataset generates trajectories with shape [Bx2x...]
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2).prefetch(3)

dataset

In [ ]:
iterator = iter(dataset)
print(iterator)

In [ ]:
# For the curious:
# Uncomment to see what the dataset iterator is feeding to the agent.
# Compare this representation of replay data 
# to the collection of individual trajectories shown earlier.

# iterator.next()

## 에이전트 훈련하기

훈련 루프 동안 두 가지 일이 발생해야 합니다.

- 환경에서 데이터 수집
- 이 데이터를 사용하여 에이전트의 신경망 훈련

이 예재는 정책을 정기적으로 평가하고 현재 점수를 인쇄합니다.

다음은 실행하는 데 ~5분이 걸립니다.

In [ ]:
#@test {"skip": true}
try:
  %%time
except:
  pass

# (Optional) Optimize by wrapping some of the code in a graph using TF function.
agent.train = common.function(agent.train)

# Reset the train step.
agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

# Reset the environment.
time_step = train_py_env.reset()

# Create a driver to collect experience.
collect_driver = py_driver.PyDriver(
    env,
    py_tf_eager_policy.PyTFEagerPolicy(
      agent.collect_policy, use_tf_function=True),
    [rb_observer],
    max_steps=collect_steps_per_iteration)

for _ in range(num_iterations):

  # Collect a few steps and save to the replay buffer.
  time_step, _ = collect_driver.run(time_step)

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience).loss

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    returns.append(avg_return)

## 시각화


### 플롯

`matplotlib.pyplot`을 사용하여 훈련 중에 정책이 얼마나 개선되었는지 차트로 표시합니다.

`Cartpole-v0`에서 한 번의 반복은 200개 타임스텝으로 구성됩니다. 환경은 막대가 위로 유지되는 각 단계에 대해 `+1`의 보상을 제공하므로 한 에피소드의 최대 이익은 200입니다. 차트는 훈련 중에 평가될 때마다 최대 값으로 증가하는 이익을 보여줍니다. (매번 불안정하고 일정하게 증가하지 않을 수 있습니다.)

In [ ]:
#@test {"skip": true}

iterations = range(0, num_iterations + 1, eval_interval)
plt.plot(iterations, returns)
plt.ylabel('Average Return')
plt.xlabel('Iterations')
plt.ylim(top=250)

### 비디오

차트가 훌륭합니다. 그러나 더 흥미로운 것은 에이전트가 실제로 환경에서 작업을 수행하는 것을 보는 것입니다.

먼저 노트북에 비디오를 포함하는 함수를 만듭니다.

In [ ]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

이제 에이전트와 함께 Cartpole 게임의 몇 가지 에피소드를 반복합니다. 기본 Python 환경(TensorFlow 환경 래퍼의 "내부")은 환경 상태의 이미지를 출력하는 `render()` 메서드를 제공합니다. 이미지는 비디오로 수집될 수 있습니다.

In [ ]:
def create_policy_eval_video(policy, filename, num_episodes=5, fps=30):
  filename = filename + ".mp4"
  with imageio.get_writer(filename, fps=fps) as video:
    for _ in range(num_episodes):
      time_step = eval_env.reset()
      video.append_data(eval_py_env.render())
      while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = eval_env.step(action_step.action)
        video.append_data(eval_py_env.render())
  return embed_mp4(filename)

create_policy_eval_video(agent.policy, "trained-agent")

재미를 위해, 훈련된 에이전트(위)를 무작위로 이동하는 에이전트와 비교해 보세요.

In [ ]:
create_policy_eval_video(random_policy, "random-agent")